In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.0 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"          "cohort.sql"          "icd9.sql"           
[4] "lab_tests.sql"       "lab_tests_taken.sql" "population.sql"     
[7] "vital_signs.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "basics.sql")) %>%
    make_view_sql("basics", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from basics limit 10;
"
dbGetQuery(con, sql)

icustay_id,hadm_id,subject_id,age,echo,gender,weight,saps,sofa,elix_score,vent,vaso,weekday
200003,163557,27513,48.29627,1,M,77.0,24,6,0,1,1,5
200014,127229,9514,84.73004,1,M,62.0,23,3,3,1,0,1
200030,165426,14987,54.19308,1,M,113.6,17,7,19,0,1,5
200033,198650,56369,67.14657,0,M,74.0,15,1,9,0,0,2
200036,159243,9960,74.93578,0,M,79.0,20,8,16,0,0,0
200075,132255,67800,83.64320,1,F,58.1,21,8,15,1,1,0
200087,124231,61871,64.32851,1,M,95.1,20,7,13,1,1,2
200102,167826,11228,46.43237,1,M,155.0,22,15,8,1,1,3
200131,187834,47250,69.31159,0,F,101.0,20,3,0,1,1,3
200140,193876,27874,68.02678,1,M,80.0,24,6,11,1,1,3


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE